In [1]:
import DataLoader
import importlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import CoverageHelper
importlib.reload(DataLoader)
%matplotlib inline

Number of apps: 
412


Number of apps: 
412


In [2]:
test_graph_df = DataLoader.load_csv_test_graphs()
# stat_graph_df = DataLoader.load_csv_stat_graph("")
# stat_trans_graph_df = DataLoader.load_csv_stat_graph("-transitive")
stat_trans_act_graph_df = DataLoader.load_csv_stat_graph("-act-transitive")

Total entries:
2067


Total entries:
41163


In [3]:
test_graph_df.head()


,source,target,action,widget_id,widget,strategy,app
0,com.trentapps.cel.MainActivity,com.trentapps.cel.Acronymns,back,NaN,NaN,systematic,com.trentapps.cel
1,com.trentapps.cel.MainActivity,com.trentapps.cel.Acronymns,openMenu,NaN,NaN,systematic,com.trentapps.cel
2,com.trentapps.cel.MainActivity,com.trentapps.cel.Acronymns,click,2.131493e+09,android.widget.Button,systematic,com.trentapps.cel
3,com.trentapps.cel.MainActivity,com.trentapps.cel.Acronymns,click,2.131493e+09,android.widget.Button,random,com.trentapps.cel
4,com.startapp.android.publish.OverlayActivity,com.startapp.android.publish.FullScreenActivity,back,NaN,NaN,systematic,com.snaptattooadder


In [7]:
wtg_edges = stat_trans_act_graph_df.groupby("app").count().target
wtg_edges.head()

# l = stat_trans_act_graph_df[stat_trans_act_graph_df.app == "bestfreelivewallpapers.funny_photo_editor"]
# l.drop_duplicates().count()
wtg_edges = wtg_edges.drop("com.yinzcam.nfl.packers")
wtg_edges.describe()
# wtg_edges.sort_values(ascending=False)

count     375.000000
mean       84.373333
std       436.925971
min         0.000000
25%         1.000000
50%         8.000000
75%        32.500000
max      7260.000000
Name: target, dtype: float64

In [8]:
wtg_screens = stat_trans_act_graph_df[['app', 'source', 'target']].melt(
    id_vars=['app']).groupby("app").value.nunique()
wtg_screens = wtg_screens.drop("com.yinzcam.nfl.packers")
# wtg_screens = wtg_screens.drop("com.team.invilabs.how.to.know.my.future")
wtg_screens.describe()

count    375.000000
mean      12.293333
std       20.536503
min        1.000000
25%        2.000000
50%        6.000000
75%       15.500000
max      262.000000
Name: value, dtype: float64

In [9]:
test_graph_df[test_graph_df.strategy == "random"].app.nunique()
p = test_graph_df[~test_graph_df.app.isin(stat_trans_act_graph_df.app)]
p.app.nunique()
# stat_trans_act_graph_df[stat_trans_act_graph_df.app == "com.littlebithackers.babyns"]

36

In [10]:
unique_widgets_per_app = test_graph_df.groupby(["strategy", "app"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count")

unique_widgets_per_app.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="5" valign="top">random</th>
      <th rowspan="2" valign="top">alex.map.maps</th>
      <th>NaN</th>
      <td>3</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
    </tr>
    <tr>
      <th>androidlab.carlog</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
    <tr>
      <th>appinventor.ai_agiapplications.CountYourSteps</th>
      <th>android.widget.Button</th>
      <td>1</td>
    </tr>
    <tr>
      <th>asp.aniemo.pack6.ext</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
  </tbody>
</table>
</div>

In [11]:
unique_widgets_per_app['percentage'] = unique_widgets_per_app.groupby(level=0).apply(lambda x: 100 * x / x.sum())
unique_widgets_per_app.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
      <th>percentage</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="5" valign="top">random</th>
      <th rowspan="2" valign="top">alex.map.maps</th>
      <th>NaN</th>
      <td>3</td>
      <td>0.249792</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
      <td>0.166528</td>
    </tr>
    <tr>
      <th>androidlab.carlog</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.083264</td>
    </tr>
    <tr>
      <th>appinventor.ai_agiapplications.CountYourSteps</th>
      <th>android.widget.Button</th>
      <td>1</td>
      <td>0.083264</td>
    </tr>
    <tr>
      <th>asp.aniemo.pack6.ext</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.083264</td>
    </tr>
  </tbody>
</table>
</div>

In [12]:
number_of_unique_screens_per_app = test_graph_df[['strategy', 'app', 'source', 'target', 'widget']].melt(
    id_vars=['strategy', 'app', 'widget'])
number_of_unique_screens_per_app.head()

,strategy,app,widget,variable,value
0,systematic,com.trentapps.cel,NaN,source,com.trentapps.cel.MainActivity
1,systematic,com.trentapps.cel,NaN,source,com.trentapps.cel.MainActivity
2,systematic,com.trentapps.cel,android.widget.Button,source,com.trentapps.cel.MainActivity
3,random,com.trentapps.cel,android.widget.Button,source,com.trentapps.cel.MainActivity
4,systematic,com.snaptattooadder,NaN,source,com.startapp.android.publish.OverlayActivity


Edges from

In [13]:
systematic_test_graph = test_graph_df[test_graph_df.strategy == "systematic"]
systematic_test_graph.head()

,source,target,action,widget_id,widget,strategy,app
0,com.trentapps.cel.MainActivity,com.trentapps.cel.Acronymns,back,NaN,NaN,systematic,com.trentapps.cel
1,com.trentapps.cel.MainActivity,com.trentapps.cel.Acronymns,openMenu,NaN,NaN,systematic,com.trentapps.cel
2,com.trentapps.cel.MainActivity,com.trentapps.cel.Acronymns,click,2.131493e+09,android.widget.Button,systematic,com.trentapps.cel
4,com.startapp.android.publish.OverlayActivity,com.startapp.android.publish.FullScreenActivity,back,NaN,NaN,systematic,com.snaptattooadder
5,com.startapp.android.publish.OverlayActivity,com.startapp.android.publish.FullScreenActivity,back,NaN,NaN,systematic,com.snaptattooadder


In [14]:
random_test_graph = test_graph_df[(test_graph_df.strategy == "random")]
random_test_graph.head()


,source,target,action,widget_id,widget,strategy,app
3,com.trentapps.cel.MainActivity,com.trentapps.cel.Acronymns,click,2.131493e+09,android.widget.Button,random,com.trentapps.cel
10,com.snaptattooadder.FullScreenViewActivity,com.snaptattooadder.MainActivity,click,2.131296e+09,android.widget.Button,random,com.snaptattooadder
11,com.snaptattooadder.MainActivity,com.snaptattooadder.FullScreenViewActivity,click,2.131296e+09,android.widget.ImageButton,random,com.snaptattooadder
12,com.snaptattooadder.MainActivity,com.startapp.android.publish.FullScreenActivity,back,NaN,NaN,random,com.snaptattooadder
13,com.snaptattooadder.MainActivity,com.startapp.android.publish.list3d.List3DActi...,back,NaN,NaN,random,com.snaptattooadder


Systematic:

In [33]:
sys_direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, systematic_test_graph)
sys_direct_edges_not_found = sys_direct_edges_not_found[~sys_direct_edges_not_found.target.isnull()]
# unique_widgets_per_app = CoverageHelper.widget_counter(direct_edges_not_found)
# unique_widgets_per_app.head()

sys_unique_widgets_per_app = sys_direct_edges_not_found.widget.value_counts(dropna=False).to_frame(
    name="widget_count")
sys_unique_widgets_per_app["percentage"] = sys_unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
sys_unique_widgets_per_app.head()

,widget_count,percentage
NaN,1219,35.353828
android.view.View,266,7.714617
android.view.MenuItem,265,7.685615
android.widget.ImageView,199,5.771462
android.widget.Button,198,5.742459


In [36]:
sys_unique_widgets_per_app = sys_unique_widgets_per_app[~sys_unique_widgets_per_app.index.isnull()]
sys_unique_widgets_per_app['percentage'] = sys_unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
sys_unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False).iloc[:5].to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n{} &  widget\\_count &  percentage \\\\\n\\midrule\nandroid.view.View        &           266 &   11.933603 \\\\\nandroid.view.MenuItem    &           265 &   11.888739 \\\\\nandroid.widget.ImageView &           199 &    8.927770 \\\\\nandroid.widget.Button    &           198 &    8.882907 \\\\\nandroid.widget.ListView  &           186 &    8.344549 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [39]:
sys_nan_direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, systematic_test_graph)
sys_nan_direct_edges_not_found = sys_nan_direct_edges_not_found[~sys_nan_direct_edges_not_found.target.isnull()]
sys_nan_direct_edges_not_found = sys_nan_direct_edges_not_found[sys_nan_direct_edges_not_found.widget.isnull()]
sys_nan_unique_widgets_per_app = sys_nan_direct_edges_not_found.groupby(["method"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count")
sys_nan_unique_widgets_per_app["percentage"] = sys_nan_unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
sys_nan_unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False).widget_count.sum()

1219

Random:

In [51]:
ran_direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, random_test_graph)
ran_direct_edges_not_found = ran_direct_edges_not_found[~ran_direct_edges_not_found.target.isnull()]
ran_unique_widgets_per_app = ran_direct_edges_not_found.widget.value_counts(dropna=False).to_frame(
    name="widget_count")
ran_unique_widgets_per_app["percentage"] = ran_unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
ran_unique_widgets_per_app.widget_count.sum()

4118

In [52]:
ran_unique_widgets_per_app = ran_unique_widgets_per_app[~ran_unique_widgets_per_app.index.isnull()]
ran_unique_widgets_per_app['percentage'] = ran_unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
ran_unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False).widget_count.sum()

2799

In [53]:
ran_nan_direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, random_test_graph)
ran_nan_direct_edges_not_found = ran_nan_direct_edges_not_found[~ran_nan_direct_edges_not_found.target.isnull()]
ran_nan_direct_edges_not_found = ran_nan_direct_edges_not_found[ran_nan_direct_edges_not_found.widget.isnull()]
ran_nan_unique_widgets_per_app = ran_nan_direct_edges_not_found.groupby(["method"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count")
ran_nan_unique_widgets_per_app["percentage"] = ran_nan_unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
ran_nan_unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False).widget_count.sum()

1319